In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

대화 내용을 벡터스토어 라는 데이터베이스에 저장하고 나중에 조회를 해볼 수 있는 기능.

벡터 스토어에 메모리를 저장하고 호출될 때마다 가장 '눈에 띄는' 상위 K개의 문서를 쿼리 합니다.
시간 순서를 고려하는 것이 아니라 과거의 대화 내용을 검색을 통해서 필요한 내용을 가져온다. 

In [2]:
import faiss
from langchain_openai import OpenAIEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS

In [3]:
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")     # 임베딩 모델을 정의합니다.

In [4]:
# Vector Store 를 초기화 합니다.
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})

In [5]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [6]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import VectorStoreRetrieverMemory

In [7]:
llm = ChatOpenAI(api_key=key, model_name="gpt-4o-mini", temperature=0)

In [8]:
# ConversationChain을 생성합니다. 
conversation = ConversationChain(
    llm=llm,                            
    memory=VectorStoreRetrieverMemory(retriever=retriever)
)

C:\Users\lee\AppData\Local\Temp\ipykernel_2868\1877807336.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=VectorStoreRetrieverMemory(retriever=retriever)
C:\Users\lee\AppData\Local\Temp\ipykernel_2868\1877807336.py:2: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


In [9]:
conversation.predict(    
    input="안녕하세요, 오늘 면접에 참석해주셔서 감사합니다. 자기소개 부탁드립니다."
)

'안녕하세요! 면접에 초대해 주셔서 감사합니다. 저는 AI 언어 모델로, 다양한 주제에 대해 대화하고 정보를 제공하는 데 특화되어 있습니다. 제 이름은 ChatGPT이며, OpenAI에서 개발되었습니다. 저는 여러 언어를 이해하고, 질문에 답하거나, 창의적인 글을 작성하는 등 다양한 작업을 수행할 수 있습니다. 제 목표는 여러분과의 대화를 통해 유용하고 흥미로운 정보를 제공하는 것입니다. 여러분에 대해 더 알고 싶습니다! 어떤 질문이든지 편하게 해주세요.'

In [10]:
conversation.predict(    
    input="프로젝트에서 어떤 역할을 맡았나요?"
)

'저는 프로젝트에서 주로 정보 제공과 문제 해결을 담당하는 역할을 맡고 있습니다. 예를 들어, 팀원들이 필요로 하는 데이터나 자료를 찾아주거나, 특정 주제에 대한 배경 지식을 제공하여 의사결정을 돕습니다. 또한, 창의적인 아이디어를 제안하거나, 문서 작성 및 편집을 지원하기도 합니다. 제가 맡은 역할은 주로 팀의 효율성을 높이고, 프로젝트의 목표를 달성하는 데 기여하는 것입니다. 여러분의 프로젝트에서는 어떤 역할을 맡고 계신가요?'

In [11]:
conversation.predict(    
    input="팀 프로젝트에서 어려움을 겪었던 경험이 있다면 어떻게 해결했나요?"
)

'팀 프로젝트에서 어려움을 겪었던 경험이 몇 가지 있습니다. 예를 들어, 한 번은 팀원 간의 의사소통이 원활하지 않아 프로젝트 진행이 지연된 적이 있었습니다. 이 문제를 해결하기 위해 저는 팀 회의를 제안했습니다. 회의에서는 각자의 진행 상황과 어려움을 공유하고, 서로의 의견을 듣는 시간을 가졌습니다. 그 결과, 팀원들이 서로의 역할을 더 잘 이해하게 되었고, 협력할 수 있는 방법을 찾을 수 있었습니다.\n\n또 다른 경험으로는, 특정 기술적인 문제로 인해 프로젝트가 막혔던 경우가 있었습니다. 이때는 제가 관련 자료를 조사하고, 온라인 포럼이나 커뮤니티에서 유사한 문제를 겪었던 사람들의 해결책을 찾아보았습니다. 그 정보를 팀원들과 공유하고, 함께 brainstorming을 통해 해결책을 도출해냈습니다. 이런 경험들은 팀워크의 중요성을 다시 한번 깨닫게 해주었고, 문제를 함께 해결하는 과정에서 팀원 간의 유대감도 더욱 깊어졌습니다. 여러분은 팀 프로젝트에서 어떤 어려움을 겪으셨나요?'

In [12]:
# 다음의 질문을 했을 때 Vector Store 로 부터 1개(k=1 이기 때문)의 가장 관련성 높은 대화를 반환합니다.
# 질문: "면접자 전공은 무엇인가요?"

# 메모리에 질문을 통해 가장 연관성 높은 1개 대화를 추출합니다.
print(conversation.memory.load_memory_variables({"prompt": "면접자 전공은 무엇인가요?"})["history"])

input: 안녕하세요, 오늘 면접에 참석해주셔서 감사합니다. 자기소개 부탁드립니다.
response: 안녕하세요! 면접에 초대해 주셔서 감사합니다. 저는 AI 언어 모델로, 다양한 주제에 대해 대화하고 정보를 제공하는 데 특화되어 있습니다. 제 이름은 ChatGPT이며, OpenAI에서 개발되었습니다. 저는 여러 언어를 이해하고, 질문에 답하거나, 창의적인 글을 작성하는 등 다양한 작업을 수행할 수 있습니다. 제 목표는 여러분과의 대화를 통해 유용하고 흥미로운 정보를 제공하는 것입니다. 여러분에 대해 더 알고 싶습니다! 어떤 질문이든지 편하게 해주세요.


In [13]:
# 이번에는 다른 질문을 통해 가장 연관성 높은 1개 대화를 추출합니다.
# 질문: "면접자가 프로젝트에서 맡은 역할은 무엇인가요?"

print(conversation.memory.load_memory_variables({"human": "면접자가 프로젝트에서 맡은 역할은 무엇인가요?"})["history"])

input: 프로젝트에서 어떤 역할을 맡았나요?
response: 저는 프로젝트에서 주로 정보 제공과 문제 해결을 담당하는 역할을 맡고 있습니다. 예를 들어, 팀원들이 필요로 하는 데이터나 자료를 찾아주거나, 특정 주제에 대한 배경 지식을 제공하여 의사결정을 돕습니다. 또한, 창의적인 아이디어를 제안하거나, 문서 작성 및 편집을 지원하기도 합니다. 제가 맡은 역할은 주로 팀의 효율성을 높이고, 프로젝트의 목표를 달성하는 데 기여하는 것입니다. 여러분의 프로젝트에서는 어떤 역할을 맡고 계신가요?
